In [1]:
import polars as pl
import numpy as np

data_path="db_dump/uncompressed/"
title_ratings = pl.scan_csv(data_path+"title.ratings.tsv", quote_char=None, has_header=True, separator="\t", null_values="\\N")
title_basics = pl.scan_csv(data_path+"title.basics.tsv", quote_char=None, has_header=True, separator="\t", null_values="\\N")

tmp_title_ratings=title_ratings.sort("numVotes", descending=True)
tmp_title_ratings=tmp_title_ratings.with_columns(pl.col("numVotes").cumsum().alias("votes_cumsum"))

In [2]:
max=tmp_title_ratings.select(pl.max("votes_cumsum")).collect().to_numpy()
max=max[0][0]

In [3]:
tmp_title_ratings=tmp_title_ratings.collect()

In [4]:
id_list=[]
for i in range(10000):
    random_num = np.random.randint(0, max)

    # find the index of the row in which the random number fits in
    index = tmp_title_ratings.select(pl.col("votes_cumsum").search_sorted(random_num, side='right')).to_numpy()[0][0]
    title_id = tmp_title_ratings.row(index)[0]
    id_list.append(title_id)

In [5]:
title_basics_list=[]
for id in id_list:
    tmp_dict=title_basics.select([pl.col("tconst"), pl.col("primaryTitle")]).filter(pl.col("tconst")==id).first().collect().to_dict()
    title_basics_list.append(tmp_dict["primaryTitle"][0])

In [8]:
import pandas as pd
df=pd.DataFrame()
df["tconst"]=id_list
df["primaryTitle"]=title_basics_list
df.to_csv("testing/testing_query_pool.csv",header=False, sep=";", index=False)